In [1]:
import pandas as pd
import numpy as np

from hydra.utils import instantiate
from hydra import compose, initialize
from hydra.core.global_hydra import GlobalHydra

from data_pipeline.pipelinesteps import data_splitter
from model.modelorchastrator import ModelOrchestrator
from data_pipeline.dataset import Dataset
from evaluate.metric_eval import ModelEvaluator

(CVXPY) Mar 27 02:43:13 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Mar 27 02:43:13 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
 2024-03-27 14:43:13,840 -  INFO  	: my_logger.custom_logger - Logger is set up.


In [2]:
# Create config
GlobalHydra.instance().clear()
initialize(config_path="../../conf", version_base=None)
cfg = compose(config_name="config")

In [3]:
# Create Dataset
df = pd.read_feather("../../data/parquet_files/train/processed_train.feather")
df.pop('date_decision')
data_pipeline = instantiate(cfg.data_pipeline)
df = data_pipeline.apply(df)  

dataset = Dataset(data=df, data_splitter=data_splitter, target_column='target')

In [4]:
model_orchestrator = ModelOrchestrator(cfg)

model = model_orchestrator.modelpipeline
ME = ModelEvaluator(model[-1], ['roc_auc_ovr', 'f1'])

{'FeatureSelector': {'_target_': 'model.feature_selector.FeatureSelector', 'features': '${features.all_features}'}}
('FeatureSelector', {'_target_': 'model.feature_selector.FeatureSelector', 'features': '${features.all_features}'})
{'OptBinningTransformer': {'_target_': 'model.modelorchastrator.OptBinningTransformer', 'column': '${features.cat_features}'}}
('OptBinningTransformer', {'_target_': 'model.modelorchastrator.OptBinningTransformer', 'column': '${features.cat_features}'})
{'HistBooster': {'_target_': 'model.modelorchastrator.HistBooster', 'learning_rate': 0.1, 'verbose': 0, 'class_weight': 'balanced', 'max_iter': 1000, 'l2_regularization': 0.3, 'early_stopping': True, 'max_leaf_nodes': 100}}
('HistBooster', {'_target_': 'model.modelorchastrator.HistBooster', 'learning_rate': 0.1, 'verbose': 0, 'class_weight': 'balanced', 'max_iter': 1000, 'l2_regularization': 0.3, 'early_stopping': True, 'max_leaf_nodes': 100})


In [5]:
from omegaconf import DictConfig, OmegaConf
t = cfg.model.model_steps[1]
next(iter(t.items()))[1]

{'_target_': 'model.modelorchastrator.OptBinningTransformer', 'column': '${features.cat_features}'}

In [6]:
df['education_927M_last']  

0          a55475b1
1          a55475b1
2          a55475b1
3          a55475b1
4          a55475b1
             ...   
1526654    a55475b1
1526655    a55475b1
1526656    a55475b1
1526657    a55475b1
1526658    a55475b1
Name: education_927M_last, Length: 1526659, dtype: object

In [7]:
model.fit(dataset.X_train, dataset.y_train)

 2024-03-27 14:43:17,403 -  INFO  	: my_logger.custom_logger - Selecting features from Dataset


c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)


CustomModelPipeline(steps=[('0',
                            FeatureSelector(features={'cardtype_51L_last',
                                                      'childnum_185L_mean',
                                                      'classificationofcontr_400M_mode',
                                                      'collater_valueofguarantee_876L_mean',
                                                      'credacc_cards_status_52L_last',
                                                      'credacc_cards_status_52L_mode',
                                                      'credtype_322L_last',
                                                      'education_927M_last',
                                                      'familystate_447L_last',
                                                      'housetype_905L_last',
                                                      'last180dayaveragebalance_...
                            OptBinningTransformer(column=['housetype_905L_last',
                                                          'education_927M_last',
                                                          'credacc_cards_status_52L_mode',
                                                          'familystate_447L_last',
                                                          'credacc_cards_status_52L_last',
                                                          'classificationofcontr_400M_mode',
                                                          'credtype_322L_last',
                                                          'cardtype_51L_last'])),
                           ('2',
                            HistBooster(class_weight='balanced',
                                        early_stopping=True,
                                        l2_regularization=0.3, max_iter=1000,
                                        max_leaf_nodes=100))])

In [8]:
model.transform_without_predictor(dataset.X_train)

 2024-03-27 14:43:30,290 -  INFO  	: my_logger.custom_logger - Selecting features from Dataset


c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)


,overdueamount_31A_max,familystate_447L_last,credacc_cards_status_52L_last,pmts_overdue_1140A_max,classificationofcontr_400M_mode,collater_valueofguarantee_876L_mean,last180dayaveragebalance_704A_max,cardtype_51L_last,education_927M_last,numinstlswithdpd5_4187116L_last,...,residualamount_488A_max,credtype_322L_last,childnum_185L_mean,numberofcontrsvalue_358L_max,totaldebtoverduevalue_178A_max,last180dayaveragebalance_704A_mean,last180dayaveragebalance_704A_last,housetype_905L_last,pmts_overdue_1140A_min,nominalrate_281L_min
221249,NaN,0.0,0.0,0.000,0.000000,NaN,NaN,0.000000,2.220446e-16,NaN,...,NaN,-0.003175,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
1220007,0.0,0.0,0.0,3254.518,-0.000166,0.000000,NaN,0.000000,2.220446e-16,9.0,...,0.0,-0.003175,NaN,8.0,0.0,NaN,NaN,0.0,0.0,NaN
1168023,0.0,0.0,0.0,0.000,-0.000166,1250.000000,NaN,0.000000,2.220446e-16,0.0,...,NaN,-0.003175,NaN,12.0,0.0,NaN,NaN,0.0,0.0,39.0
145230,0.0,0.0,0.0,0.000,-0.000166,0.000000,NaN,0.000000,2.220446e-16,0.0,...,0.0,-0.083811,NaN,4.0,0.0,NaN,NaN,0.0,0.0,42.0
338605,NaN,0.0,0.0,NaN,0.000000,NaN,NaN,0.120749,2.220446e-16,NaN,...,NaN,0.122728,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222458,NaN,0.0,0.0,4585.916,0.000000,NaN,NaN,0.000000,2.220446e-16,NaN,...,NaN,-0.003175,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
138401,0.0,0.0,0.0,0.000,-0.000166,19866.545455,NaN,0.120749,2.220446e-16,1.0,...,0.0,0.122728,NaN,8.0,0.0,NaN,NaN,0.0,0.0,4.8
183208,0.0,0.0,0.0,3559.084,-0.000166,0.000000,NaN,0.000000,2.220446e-16,0.0,...,NaN,-0.083811,NaN,1.0,0.0,NaN,NaN,0.0,0.0,NaN
222110,NaN,0.0,0.0,NaN,0.000000,NaN,NaN,0.000000,2.220446e-16,NaN,...,NaN,-0.003175,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [9]:
ME.evaluate(model.transform_without_predictor(dataset.X_train), dataset.y_train)

 2024-03-27 14:43:31,733 -  INFO  	: my_logger.custom_logger - Selecting features from Dataset


c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)


{'fit_time': array([11.5026381 ,  9.26293111,  9.50804377, 10.59243727, 11.46250033]),
 'score_time': array([1.65061188, 1.23344684, 1.52383113, 1.35549593, 1.50056219]),
 'test_roc_auc_ovr': array([0.73983643, 0.74357917, 0.74122591, 0.73811359, 0.73851501]),
 'test_f1': array([0.11740934, 0.11763602, 0.1162018 , 0.11427294, 0.11533625])}

In [10]:
from sklearn.metrics import get_scorer_names
all_scorers = get_scorer_names()
all_scorers

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_negative_likelihood_ratio',
 'neg_root_mean_squared_error',
 'neg_root_mean_squared_log_error',
 'normalized_mutual_info_score',
 'positive_likelihood_ratio',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 '

In [11]:
from sklearn.metrics import roc_auc_score

def gini(y_true, y_pred):
    return 2 * roc_auc_score(y_true, y_pred) - 1

In [12]:
gini(dataset.y_train, model.predict_proba(dataset.X_train).T[1])



 2024-03-27 14:44:32,618 -  INFO  	: my_logger.custom_logger - Selecting features from Dataset


c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)


0.5416508946083336

In [13]:
roc_auc_score(dataset.y_train, model.predict_proba(dataset.X_train).T[1])

 2024-03-27 14:44:38,098 -  INFO  	: my_logger.custom_logger - Selecting features from Dataset


c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)


0.7708254473041668

In [14]:
roc_auc_score(dataset.y_oot, model.predict_proba(dataset.X_oot).T[1])

 2024-03-27 14:44:43,114 -  INFO  	: my_logger.custom_logger - Selecting features from Dataset


c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)


0.7070231655025356